<a href="https://colab.research.google.com/github/adas-quant/mscfe-gwp1-yieldcurve/blob/arindam-task2/notebooks/02_Task2_YieldCurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2 — Yield Curve Modeling
Initial skeleton notebook
